In [1]:
import pandas as pd 
from tqdm import tqdm
from konlpy.tag import Okt, Kkma
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np 
from tqdm import tqdm

In [2]:
# 파일 불러오기
data = pd.read_csv('./data_2.csv')
del data['Unnamed: 0']
data.head()

,관광지,주소,kr_text,total_1000,nouns,top10,top1000,latitude,longitude
0,경복궁,서울특별시 종로구 사직로 161(경복궁),은우맘의 취미생활광화문 역 맛집 엄마와 한정식 점심 꽃누리들밥 점오랜만에 역 근처...,"[('맛집', 403), ('서울', 250), ('야간', 189), ('카페',...",은우 취미 생활 광화문 맛집 엄마 한정식 점심 누리 근처 약속 양식 때문 한정식 ...,맛집 서울 야간 카페 개장 시간 광화문 한복 관람 근처,맛집 서울 야간 카페 개장 시간 광화문 한복 관람 근처 가을 여행 데이트 위치 삼...,37.579617,126.977041
1,창덕궁,서울특별시 종로구 율곡로 99(창덕궁),동에번쩍 서에번쩍 구길동 서울궁투어 후원 예약방법 전각 입장료 주차장꿀팁내돈내산...,"[('후원', 288), ('서울', 281), ('카페', 219), ('창경궁'...",번쩍 번쩍 길동 서울 투어 후원 예약 방법 전각 입장료 주차장 꿀팁 서울 투어 후...,후원 서울 카페 창경궁 관람 궁궐 달빛 여행 기행 경복궁,후원 서울 카페 창경궁 관람 궁궐 달빛 여행 기행 경복궁 시간 고궁 매화 안국역 ...,37.581122,126.991112
2,창경궁,서울특별시 종로구 창경궁로 185(창경궁),사나의 여행 대온실 서울 궁궐 데이트 주차장 정보까지 대온실 생각이 딱 나더라고요...,"[('서울', 290), ('창덕궁', 258), ('궁궐', 178), ('야간'...",사나 여행 온실 서울 궁궐 데이트 주차장 정보 온실 생각 외관 감탄 역사 흔적 온...,서울 창덕궁 궁궐 야간 온실 개장 여행 시간 고궁 스냅,서울 창덕궁 궁궐 야간 온실 개장 여행 시간 고궁 스냅 한복 경복궁 문화 나들이 ...,37.578771,126.994870
3,덕수궁,서울특별시 종로구 세종대로 99(덕수궁),일 전 서울 한정식 맛집 한가람 본점 시청 점심서울 한정식 맛집 시청 점심 ...,"[('서울', 394), ('미술관', 197), ('돌담', 189), ('여행'...",서울 한정식 맛집 한가람 본점 시청 점심 서울 한정식 맛집 시청 점심 한가람 본점...,서울 미술관 돌담 여행 맛집 정동 장욱진 석조전 국립 카페,서울 미술관 돌담 여행 맛집 정동 장욱진 석조전 국립 카페 현대 와플 궁궐 시간 ...,37.565805,126.975146
4,경희궁,서울특별시 종로구 새문안로 55(경희궁),장돌뱅이와 곱단이의 살아가는 이야기정조는 지에 이렇게 썼다 대체로 궁궐이란 임금이...,"[('서울', 267), ('맛집', 254), ('박물관', 198), ('떡볶이...",장돌 뱅이 이의 이야기 정조 대체로 궁궐 임금 거처 정치 사방 우러러 채색 덕수궁...,서울 맛집 박물관 떡볶이 광화문 단지 궁궐 종로구 이점 시간,서울 맛집 박물관 떡볶이 광화문 단지 궁궐 종로구 이점 시간 서울역 카페 위치 종...,37.570755,126.968834


## 위도, 경도 변환 - GEO 코딩

In [3]:
from geopy.geocoders import Nominatim

In [4]:
geo_local = Nominatim(user_agent='South Korea')

# 위도, 경도 반환하는 함수
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]


In [5]:
latitude = []
longitude =[]

for i in tqdm(data['주소']):
    latitude.append(geocoding(i)[0])
    longitude.append(geocoding(i)[1])

100%|████████████████████████████████████████████████████████████████████████████████| 122/122 [02:06<00:00,  1.04s/it]


In [6]:
data['latitude'] = latitude
data['longitude'] = longitude
data.head()

,관광지,주소,kr_text,total_1000,nouns,top10,top1000,latitude,longitude
0,경복궁,서울특별시 종로구 사직로 161(경복궁),은우맘의 취미생활광화문 역 맛집 엄마와 한정식 점심 꽃누리들밥 점오랜만에 역 근처...,"[('맛집', 403), ('서울', 250), ('야간', 189), ('카페',...",은우 취미 생활 광화문 맛집 엄마 한정식 점심 누리 근처 약속 양식 때문 한정식 ...,맛집 서울 야간 카페 개장 시간 광화문 한복 관람 근처,맛집 서울 야간 카페 개장 시간 광화문 한복 관람 근처 가을 여행 데이트 위치 삼...,37.575792,126.973629
1,창덕궁,서울특별시 종로구 율곡로 99(창덕궁),동에번쩍 서에번쩍 구길동 서울궁투어 후원 예약방법 전각 입장료 주차장꿀팁내돈내산...,"[('후원', 288), ('서울', 281), ('카페', 219), ('창경궁'...",번쩍 번쩍 길동 서울 투어 후원 예약 방법 전각 입장료 주차장 꿀팁 서울 투어 후...,후원 서울 카페 창경궁 관람 궁궐 달빛 여행 기행 경복궁,후원 서울 카페 창경궁 관람 궁궐 달빛 여행 기행 경복궁 시간 고궁 매화 안국역 ...,37.582387,126.991701
2,창경궁,서울특별시 종로구 창경궁로 185(창경궁),사나의 여행 대온실 서울 궁궐 데이트 주차장 정보까지 대온실 생각이 딱 나더라고요...,"[('서울', 290), ('창덕궁', 258), ('궁궐', 178), ('야간'...",사나 여행 온실 서울 궁궐 데이트 주차장 정보 온실 생각 외관 감탄 역사 흔적 온...,서울 창덕궁 궁궐 야간 온실 개장 여행 시간 고궁 스냅,서울 창덕궁 궁궐 야간 온실 개장 여행 시간 고궁 스냅 한복 경복궁 문화 나들이 ...,37.580246,126.994833
3,덕수궁,서울특별시 종로구 세종대로 99(덕수궁),일 전 서울 한정식 맛집 한가람 본점 시청 점심서울 한정식 맛집 시청 점심 ...,"[('서울', 394), ('미술관', 197), ('돌담', 189), ('여행'...",서울 한정식 맛집 한가람 본점 시청 점심 서울 한정식 맛집 시청 점심 한가람 본점...,서울 미술관 돌담 여행 맛집 정동 장욱진 석조전 국립 카페,서울 미술관 돌담 여행 맛집 정동 장욱진 석조전 국립 카페 현대 와플 궁궐 시간 ...,0.000000,0.000000
4,경희궁,서울특별시 종로구 새문안로 55(경희궁),장돌뱅이와 곱단이의 살아가는 이야기정조는 지에 이렇게 썼다 대체로 궁궐이란 임금이...,"[('서울', 267), ('맛집', 254), ('박물관', 198), ('떡볶이...",장돌 뱅이 이의 이야기 정조 대체로 궁궐 임금 거처 정치 사방 우러러 채색 덕수궁...,서울 맛집 박물관 떡볶이 광화문 단지 궁궐 종로구 이점 시간,서울 맛집 박물관 떡볶이 광화문 단지 궁궐 종로구 이점 시간 서울역 카페 위치 종...,0.000000,0.000000


In [11]:
data.iloc[2:6][['관광지','주소','latitude', 'longitude']]

,관광지,주소,latitude,longitude
2,창경궁,서울특별시 종로구 창경궁로 185(창경궁),37.580246,126.994833
3,덕수궁,서울특별시 종로구 세종대로 99(덕수궁),0.000000,0.000000
4,경희궁,서울특별시 종로구 새문안로 55(경희궁),0.000000,0.000000
5,홍대거리,서울특별시 마포구,37.566571,126.901532


In [9]:
len(data[data['latitude']==0])

53

## 구글스프레드시트 지오코딩 확장프로그램 사용

In [3]:
map_=pd.read_excel('../../tour_100_list.xlsx')
map_

,관광지,주소,Latitude,Longitude
0,경복궁,서울특별시 종로구 사직로 161(경복궁),37.579617,126.977041
1,창덕궁,서울특별시 종로구 율곡로 99(창덕궁),37.581122,126.991112
2,창경궁,서울특별시 종로구 창경궁로 185(창경궁),37.578771,126.994870
3,덕수궁,서울특별시 종로구 세종대로 99(덕수궁),37.565805,126.975146
4,경희궁,서울특별시 종로구 새문안로 55(경희궁),37.570755,126.968834
...,...,...,...,...
117,한라산국립공원,제주특별자치도 제주시 제주시 1100로 2070-61,33.391774,126.494501
118,제주올레길,"제주특별자치도 제주시, 서귀포시",33.254120,126.560076
119,우도,제주특별자치도 제주시 삼양고수물길 1,33.519470,126.951072
120,비자림,제주특별자치도 제주시 구좌읍 비자숲길 55,33.491432,126.811138


In [4]:
data = pd.read_csv('../../data_2.csv')
del data['Unnamed: 0']
data

,관광지,주소,kr_text,total_1000,nouns,top10,top1000
0,경복궁,서울특별시 종로구 사직로 161(경복궁),은우맘의 취미생활광화문 역 맛집 엄마와 한정식 점심 꽃누리들밥 점오랜만에 역 근처...,"[('맛집', 403), ('서울', 250), ('야간', 189), ('카페',...",은우 취미 생활 광화문 맛집 엄마 한정식 점심 누리 근처 약속 양식 때문 한정식 ...,맛집 서울 야간 카페 개장 시간 광화문 한복 관람 근처,맛집 서울 야간 카페 개장 시간 광화문 한복 관람 근처 가을 여행 데이트 위치 삼...
1,창덕궁,서울특별시 종로구 율곡로 99(창덕궁),동에번쩍 서에번쩍 구길동 서울궁투어 후원 예약방법 전각 입장료 주차장꿀팁내돈내산...,"[('후원', 288), ('서울', 281), ('카페', 219), ('창경궁'...",번쩍 번쩍 길동 서울 투어 후원 예약 방법 전각 입장료 주차장 꿀팁 서울 투어 후...,후원 서울 카페 창경궁 관람 궁궐 달빛 여행 기행 경복궁,후원 서울 카페 창경궁 관람 궁궐 달빛 여행 기행 경복궁 시간 고궁 매화 안국역 ...
2,창경궁,서울특별시 종로구 창경궁로 185(창경궁),사나의 여행 대온실 서울 궁궐 데이트 주차장 정보까지 대온실 생각이 딱 나더라고요...,"[('서울', 290), ('창덕궁', 258), ('궁궐', 178), ('야간'...",사나 여행 온실 서울 궁궐 데이트 주차장 정보 온실 생각 외관 감탄 역사 흔적 온...,서울 창덕궁 궁궐 야간 온실 개장 여행 시간 고궁 스냅,서울 창덕궁 궁궐 야간 온실 개장 여행 시간 고궁 스냅 한복 경복궁 문화 나들이 ...
3,덕수궁,서울특별시 종로구 세종대로 99(덕수궁),일 전 서울 한정식 맛집 한가람 본점 시청 점심서울 한정식 맛집 시청 점심 ...,"[('서울', 394), ('미술관', 197), ('돌담', 189), ('여행'...",서울 한정식 맛집 한가람 본점 시청 점심 서울 한정식 맛집 시청 점심 한가람 본점...,서울 미술관 돌담 여행 맛집 정동 장욱진 석조전 국립 카페,서울 미술관 돌담 여행 맛집 정동 장욱진 석조전 국립 카페 현대 와플 궁궐 시간 ...
4,경희궁,서울특별시 종로구 새문안로 55(경희궁),장돌뱅이와 곱단이의 살아가는 이야기정조는 지에 이렇게 썼다 대체로 궁궐이란 임금이...,"[('서울', 267), ('맛집', 254), ('박물관', 198), ('떡볶이...",장돌 뱅이 이의 이야기 정조 대체로 궁궐 임금 거처 정치 사방 우러러 채색 덕수궁...,서울 맛집 박물관 떡볶이 광화문 단지 궁궐 종로구 이점 시간,서울 맛집 박물관 떡볶이 광화문 단지 궁궐 종로구 이점 시간 서울역 카페 위치 종...
...,...,...,...,...,...,...,...
117,한라산국립공원,제주특별자치도 제주시 제주시 1100로 2070-61,기억창고 어리목남벽영실어리목탐방안내소 한라산을 다시는 안 올줄 알았다 재미 없는 ...,"[('한라산', 1043), ('국립공원', 481), ('탐방', 409), ('...",기억 창고 목남벽영 실어 리목 탐방 안내소 한라산 재미 고통 보상 시간 동안 국립...,한라산 국립공원 탐방 제주 코스 제주도 사랑 오름 등산 판악,한라산 국립공원 탐방 제주 코스 제주도 사랑 오름 등산 판악 시간 백록담 여행 환...
118,제주올레길,"제주특별자치도 제주시, 서귀포시",당신의 일상은 안녕한가요서귀포 여행 제주 가파도 배시간 제주 올레길 코스 추천제주...,"[('제주', 1158), ('코스', 1140), ('올레길', 1092), ('...",당신 일상 서귀포 여행 제주 가파도 배시 제주 올레길 코스 추천 제주 올레길 추천...,제주 코스 올레길 제주도 여행 올레 서귀포 걷기 추천 시간,제주 코스 올레길 제주도 여행 올레 서귀포 걷기 추천 시간 바다 완주 오름 시작 ...
119,우도,제주특별자치도 제주시 삼양고수물길 1,어제박로우 천천히 걸어가기횟집 회양과국수군 직접 잡아 손질하는 제주도 물회 맛집...,"[('여행', 482), ('제주', 458), ('제주도', 373), ('맛집'...",어제 로우 횟집 회양 국수 직접 손질 제주도 물회 맛집 요약 회양 국수 개인 숙소...,여행 제주 제주도 맛집 땅콩 카페 아이스크림 성산 해수욕장 시간,여행 제주 제주도 맛집 땅콩 카페 아이스크림 성산 해수욕장 시간 배시 코스 버거 ...
120,비자림,제주특별자치도 제주시 구좌읍 비자숲길 55,일 전모든 일이 일어나는 데는 이유가 있어요푸르른 천년의 숲 그린카드 무료입...,"[('제주', 696), ('제주도', 392), ('여행', 358), ('숲길'...",전모 이유 천년 그린카드 무료 입장 천연기념물 지정 보호 천년 도착 주차장 매표소...,제주 제주도 여행 숲길 맛집 천년 동쪽 시간 비자나무 구좌읍,제주 제주도 여행 숲길 맛집 천년 동쪽 시간 비자나무 구좌읍 카페 코스 근처 산책...


In [5]:
data['latitude'] = map_['Latitude']
data['longitude'] = map_['Longitude']
data

,관광지,주소,kr_text,total_1000,nouns,top10,top1000,latitude,longitude
0,경복궁,서울특별시 종로구 사직로 161(경복궁),은우맘의 취미생활광화문 역 맛집 엄마와 한정식 점심 꽃누리들밥 점오랜만에 역 근처...,"[('맛집', 403), ('서울', 250), ('야간', 189), ('카페',...",은우 취미 생활 광화문 맛집 엄마 한정식 점심 누리 근처 약속 양식 때문 한정식 ...,맛집 서울 야간 카페 개장 시간 광화문 한복 관람 근처,맛집 서울 야간 카페 개장 시간 광화문 한복 관람 근처 가을 여행 데이트 위치 삼...,37.579617,126.977041
1,창덕궁,서울특별시 종로구 율곡로 99(창덕궁),동에번쩍 서에번쩍 구길동 서울궁투어 후원 예약방법 전각 입장료 주차장꿀팁내돈내산...,"[('후원', 288), ('서울', 281), ('카페', 219), ('창경궁'...",번쩍 번쩍 길동 서울 투어 후원 예약 방법 전각 입장료 주차장 꿀팁 서울 투어 후...,후원 서울 카페 창경궁 관람 궁궐 달빛 여행 기행 경복궁,후원 서울 카페 창경궁 관람 궁궐 달빛 여행 기행 경복궁 시간 고궁 매화 안국역 ...,37.581122,126.991112
2,창경궁,서울특별시 종로구 창경궁로 185(창경궁),사나의 여행 대온실 서울 궁궐 데이트 주차장 정보까지 대온실 생각이 딱 나더라고요...,"[('서울', 290), ('창덕궁', 258), ('궁궐', 178), ('야간'...",사나 여행 온실 서울 궁궐 데이트 주차장 정보 온실 생각 외관 감탄 역사 흔적 온...,서울 창덕궁 궁궐 야간 온실 개장 여행 시간 고궁 스냅,서울 창덕궁 궁궐 야간 온실 개장 여행 시간 고궁 스냅 한복 경복궁 문화 나들이 ...,37.578771,126.994870
3,덕수궁,서울특별시 종로구 세종대로 99(덕수궁),일 전 서울 한정식 맛집 한가람 본점 시청 점심서울 한정식 맛집 시청 점심 ...,"[('서울', 394), ('미술관', 197), ('돌담', 189), ('여행'...",서울 한정식 맛집 한가람 본점 시청 점심 서울 한정식 맛집 시청 점심 한가람 본점...,서울 미술관 돌담 여행 맛집 정동 장욱진 석조전 국립 카페,서울 미술관 돌담 여행 맛집 정동 장욱진 석조전 국립 카페 현대 와플 궁궐 시간 ...,37.565805,126.975146
4,경희궁,서울특별시 종로구 새문안로 55(경희궁),장돌뱅이와 곱단이의 살아가는 이야기정조는 지에 이렇게 썼다 대체로 궁궐이란 임금이...,"[('서울', 267), ('맛집', 254), ('박물관', 198), ('떡볶이...",장돌 뱅이 이의 이야기 정조 대체로 궁궐 임금 거처 정치 사방 우러러 채색 덕수궁...,서울 맛집 박물관 떡볶이 광화문 단지 궁궐 종로구 이점 시간,서울 맛집 박물관 떡볶이 광화문 단지 궁궐 종로구 이점 시간 서울역 카페 위치 종...,37.570755,126.968834
...,...,...,...,...,...,...,...,...,...
117,한라산국립공원,제주특별자치도 제주시 제주시 1100로 2070-61,기억창고 어리목남벽영실어리목탐방안내소 한라산을 다시는 안 올줄 알았다 재미 없는 ...,"[('한라산', 1043), ('국립공원', 481), ('탐방', 409), ('...",기억 창고 목남벽영 실어 리목 탐방 안내소 한라산 재미 고통 보상 시간 동안 국립...,한라산 국립공원 탐방 제주 코스 제주도 사랑 오름 등산 판악,한라산 국립공원 탐방 제주 코스 제주도 사랑 오름 등산 판악 시간 백록담 여행 환...,33.391774,126.494501
118,제주올레길,"제주특별자치도 제주시, 서귀포시",당신의 일상은 안녕한가요서귀포 여행 제주 가파도 배시간 제주 올레길 코스 추천제주...,"[('제주', 1158), ('코스', 1140), ('올레길', 1092), ('...",당신 일상 서귀포 여행 제주 가파도 배시 제주 올레길 코스 추천 제주 올레길 추천...,제주 코스 올레길 제주도 여행 올레 서귀포 걷기 추천 시간,제주 코스 올레길 제주도 여행 올레 서귀포 걷기 추천 시간 바다 완주 오름 시작 ...,33.254120,126.560076
119,우도,제주특별자치도 제주시 삼양고수물길 1,어제박로우 천천히 걸어가기횟집 회양과국수군 직접 잡아 손질하는 제주도 물회 맛집...,"[('여행', 482), ('제주', 458), ('제주도', 373), ('맛집'...",어제 로우 횟집 회양 국수 직접 손질 제주도 물회 맛집 요약 회양 국수 개인 숙소...,여행 제주 제주도 맛집 땅콩 카페 아이스크림 성산 해수욕장 시간,여행 제주 제주도 맛집 땅콩 카페 아이스크림 성산 해수욕장 시간 배시 코스 버거 ...,33.519470,126.951072
120,비자림,제주특별자치도 제주시 구좌읍 비자숲길 55,일 전모든 일이 일어나는 데는 이유가 있어요푸르른 천년의 숲 그린카드 무료입...,"[('제주', 696), ('제주도', 392), ('여행', 358), ('숲길'...",전모 이유 천년 그린카드 무료 입장 천연기념물 지정 보호 천년 도착 주차장 매표소...,제주 제주도 여행 숲길 맛집 천년 동쪽 시간 비자나무 구좌읍,제주 제주도 여행 숲길 맛집 천년 동쪽 시간 비자나무 구좌읍 카페 코스 근처 산책...,33.491432,126.811138


In [6]:
data.to_csv("data_2.csv", encoding='utf-8')